<a href="https://colab.research.google.com/github/n-goyal/Data-Extraction-and-Text-Analysis/blob/master/DataExtraction&TextAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#drive dependencies
!pip install xlrd
!pip install openpyxl
!pip install gensim
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#authenticate

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 112kB 2.5MB/s 
    100% |████████████████████████████████| 1.9MB 12.6MB/s 
  Running setup.py bdist_wheel for openpyxl ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/57/41/b9/3765af8bda4a8d4b6aaf4957d7214984c3332348713e85cf36
  Running setup.py bdist_wheel for et-xmlfile ... - done
  Stored in directory: /root/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile
    100% |████████████████████████████████| 23.6MB 1.3MB/s 
    100% |████████████████████████████████| 1.4MB 13.5MB/s 
    100% |████████████████████████████████| 133kB 28.0MB/s 
    100% |████████████████████████████████| 4.7MB 5.7MB/s 
    100% |████████████████████████████████| 61kB 21.9MB/s 
    100% |████████████████████████████████| 552kB 23.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa30

In [0]:
# downloading data from drive
data_excel = '15LcY9lrv5W7cX2HuUp1lhFbS4oxVN_2p'
downloaded = drive.CreateFile({'id': data_excel})
downloaded.GetContentFile('cik_list.xlsx')

In [0]:
# data dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
import openpyxl
import nltk
import urllib
import urllib.request
import requests
import time
# from pattern.en import ngrams

In [61]:
# reading data from excel sheet using pandas
ecl = pd.read_excel('cik_list.xlsx')
print(ecl.FORM.unique())
quarter = ecl.loc[ecl['FORM'].isin(['10-Q','NT 10-Q','10-Q/A'])]
year = ecl.loc[ecl['FORM'].isin(['10-K','NT 10-K','10-K/A', '10-K405/A', '10-K405'])]
print(quarter.shape)
print(year.shape)

['10-K405' '10-Q' 'NT 10-Q' '10-K/A' '10-Q/A' 'NT 10-K' '10-K' '10-K405/A']
(110, 6)
(42, 6)


In [6]:
# making a url list
url_begin = 'https://www.sec.gov/Archives/'
url_list = []
for i in list(ecl.SECFNAME):
  url_list.append(url_begin + i)
len(url_list)

152

In [0]:
# beautiful soup function for text extraction from web pages:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
def extraction(url):
  search = url
  home_page = requests.get(search,headers=headers)
  home_content = home_page.content
  soup = bs(home_content, 'html.parser')
  file = soup.text
  return file

In [0]:
file=[]
for i in url_list:
  text = extraction(i)
  file.append(text)

In [0]:
# making a new dataframe with complete url and extracted text for easy navigation:

data = pd.DataFrame(columns=['Data', 'URL', 'SECFNAME'])
data.Data = file
data.URL = url_list
data.SECFNAME = ecl.SECFNAME

(152, 3)


,Data,URL,SECFNAME
0,-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc...,https://www.sec.gov/Archives/edgar/data/3662/0...,edgar/data/3662/0000950170-98-000413.txt
1,-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc...,https://www.sec.gov/Archives/edgar/data/3662/0...,edgar/data/3662/0000950170-98-001001.txt
2,-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc...,https://www.sec.gov/Archives/edgar/data/3662/0...,edgar/data/3662/0000950172-98-000783.txt
3,-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc...,https://www.sec.gov/Archives/edgar/data/3662/0...,edgar/data/3662/0000950170-98-002145.txt
4,-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc...,https://www.sec.gov/Archives/edgar/data/3662/0...,edgar/data/3662/0000950172-98-001203.txt


In [53]:
# observation purpose
print(data.URL[94])
print(data.URL[149])

https://www.sec.gov/Archives/edgar/data/6071/0000006071-00-000002.txt
https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-041441.txt


In [0]:
# making a dataframe to extract required sectional data:

section_data = pd.DataFrame(columns = ['MDA', 'QAQD', 'RFA'])
MDA = []
QAQD = []
RFA = []

In [0]:
# function for clearing the HTML tags from the text for easy data extraction:

def cleanhtml(raw_html):
  cleantext = re.sub('(?s)<[^>]*>', '', raw_html)
  return cleantext

clean_files = []
for i in data.Data:
  clean_files.append(cleanhtml(i))

In [0]:
# apply function based on form type of the report

for i in data.Data:
  CONFORMED SUBMISSION TYPE = re.compile(r"CONFORMED SUBMISSION TYPE)
  if CONFORMED SUBMISSION TYPE == '10-K':
    extraction_10K(file)
  else:
    extraction_10Q(file)

In [0]:
# section extraction from file for further text analysis of the report:

def extraction_10K(file):
  patternMDA = re.compile(r"(?s)Item 7[^A].*?Item 7")
  if patternMDA.search(file) != None:
    MDA = patternMDA.search(file).group()
  else:
    MDA = None
  patternQAQD = re.compile(r"(?s)Item 7A.*?Item 8")
  if patternQAQD.search(file) != None:
    QAQD = patternQAQD.search(file).group()
  else:
    QAQD = None
  patternRFA = re.compile(r"(?s)Item 1A.*?Item 1B")
  if patternRFA.search(file) != None:
    RFA = patternRFA.search(file).group()
  else:
    RFA = None
  return MDA,QAQD,RFA

def extraction_10_Q(file):
  

In [43]:
# search for form type section in the file to extract and 
# check the form type of the form:

Type = re.compile("CONFORMED SUBMISSION TYPE:.*")
z = Type.search(data.Data[150]).group()
z

'CONFORMED SUBMISSION TYPE:\t10-K/A'

In [52]:
# check form type of the report for further extraction:
res = z.split('\t').pop();
res

# compare with available types:
if res == '10-Q':
  print('true')
else:
  print('false')

false


In [0]:
# clean HTML tags from files:
file = data.Data[149]
clean = re.sub('(?s)<[^>]*>', '', file)
# print(clean)

In [0]:
for i in data.data:
  MDA.append(extraction_10_K(i)[0])
  QAQD.append(extraction_10_K(i)[1])
  RFA.append(extraction_10_K(i)[2])

In [0]:
section_data.MDA = MDA
section_data.QAQD = QAQD
section_data.RFA = RFA